## YOLOv5 loss analysis, as performed in the medium article

In [32]:
import torch
import torch.nn as nn

from ultralytics_tools.metrics import bbox_iou
from loss import YOLOv5Loss, YOLOv3Loss

Loss calculation example

In [33]:
device = 'cpu'
img_size = 320
num_classes = 20; num_layers = 3
anchor_t = 4.0

# Loss weights
balance = [4.0, 1.0, 0.4]
lambda_box = 0.05; lambda_obj = 0.7; lambda_cls = 0.3

anchors = torch.tensor([
    # Anchor_1 for each prediction layer
    [[ 1.25000,  1.62500],[ 2.00000,  3.75000],[ 4.12500,  2.87500]], 
    # Anchor_2 for each prediction layer
    [[ 1.87500,  3.81250],[ 3.87500,  2.81250],[ 3.68750,  7.43750]], 
    # Anchor_3 for each prediction layer
    [[ 3.62500,  2.81250],[ 4.87500,  6.18750],[11.65625, 10.18750]], 
])
num_anchors = anchors.shape[0]

targets = torch.tensor([
        [ 0.00000, 14.00000,  0.49535,  0.50528,  0.15267,  0.56956],
        [ 0.00000,  0.00000,  0.54872,  0.92491,  0.05361,  0.03183],
        [ 0.00000,  0.00000,  0.36780,  0.98716,  0.06031,  0.02567],
        [ 1.00000,  6.00000,  0.97072,  0.04398,  0.05856,  0.08796],
        [ 1.00000, 16.00000,  0.70696,  0.10348,  0.32971,  0.16793],
])
batch_size = len(targets[:,:1].unique())

strides = [8, 16, 32]
p = [
    torch.randn((batch_size, num_anchors, img_size//strides[i], img_size//strides[i], 5 + num_classes)) 
    for i in range(num_layers)
]

print("Targets Shape:", targets.shape)
print("Anchors Shape:", anchors.shape)
for i, pi in enumerate(p):
    print(f"Layer P{i+3} Shape:", pi.shape)

Targets Shape: torch.Size([5, 6])
Anchors Shape: torch.Size([3, 3, 2])
Layer P3 Shape: torch.Size([2, 3, 40, 40, 25])
Layer P4 Shape: torch.Size([2, 3, 20, 20, 25])
Layer P5 Shape: torch.Size([2, 3, 10, 10, 25])


Original YOLOv5 loss function with minor adjustments

In [34]:
class ComputeLoss:
    sort_obj_iou = False

    # Compute losses
    def __init__(self, autobalance=False, anchor_t:int=4, balance:list[int]=[4.0, 1.0, 0.4], 
                 strides=[8,16,32], lambda_box=0.05, lambda_obj=0.7, lambda_cls=0.3):
        """Initializes ComputeLoss with model and autobalance option, autobalances losses if True."""
        # Define criteria
        BCEcls = nn.BCEWithLogitsLoss()
        BCEobj = nn.BCEWithLogitsLoss()

        # Class label smoothing https://arxiv.org/pdf/1902.04103.pdf eqn 3
        self.cp, self.cn = 1.0, 0.0
        
        self.balance = balance
        
        self.ssi = strides.index(16) if autobalance else 0  # stride 16 index
        self.BCEcls, self.BCEobj, self.gr, self.autobalance = BCEcls, BCEobj, 1.0, autobalance
        self.na = num_anchors  # number of anchors
        self.nc = num_classes  # number of classes
        self.nl = num_layers  # number of layers
        self.anchors = anchors
        self.device = device
        
        self.anchor_t = anchor_t
        self.lambda_box = lambda_box
        self.lambda_obj = lambda_obj
        self.lambda_cls = lambda_cls
        

    def __call__(self, p, targets):  # predictions, targets
        """Performs forward pass, calculating class, box, and object loss for given predictions and targets."""
        lcls = torch.zeros(1, device=self.device)  # class loss
        lbox = torch.zeros(1, device=self.device)  # box loss
        lobj = torch.zeros(1, device=self.device)  # object loss
        
        tcls, tbox, indices, anchors = self.build_targets(p, targets)  # targets
        
        # Losses
        for i, pi in enumerate(p):  # layer index, layer predictions
            b, a, gj, gi = indices[i]  # image, anchor, gridy, gridx
            tobj = torch.zeros(pi.shape[:4], dtype=pi.dtype, device=self.device)  # target obj
            
            n = b.shape[0]  # number of targets
            if n:
                # pxy, pwh, _, pcls = pi[b, a, gj, gi].tensor_split((2, 4, 5), dim=1)  # faster, requires torch 1.8.0
                pxy, pwh, _, pcls = pi[b, a, gj, gi].split((2, 2, 1, self.nc), 1)  # target-subset of predictions

                # Regression
                pxy = pxy.sigmoid() * 2 - 0.5
                pwh = (pwh.sigmoid() * 2) ** 2 * anchors[i]
                pbox = torch.cat((pxy, pwh), 1)  # predicted box
                iou = bbox_iou(pbox, tbox[i], CIoU=True).squeeze()  # iou(prediction, target)
                lbox += (1.0 - iou).mean()  # iou loss

                # Objectness
                iou = iou.detach().clamp(0).type(tobj.dtype)
                if self.sort_obj_iou:
                    j = iou.argsort()
                    b, a, gj, gi, iou = b[j], a[j], gj[j], gi[j], iou[j]
                if self.gr < 1:
                    iou = (1.0 - self.gr) + self.gr * iou
                tobj[b, a, gj, gi] = iou  # iou ratio

                # Classification
                if self.nc > 1:  # cls loss (only if multiple classes)
                    t = torch.full_like(pcls, self.cn, device=self.device)  # targets
                    t[range(n), tcls[i]] = self.cp
                    lcls += self.BCEcls(pcls, t)  # BCE

                # Append targets to text file
                # with open('targets.txt', 'a') as file:
                #     [file.write('%11.5g ' * 4 % tuple(x) + '\n') for x in torch.cat((txy[i], twh[i]), 1)]

            obji = self.BCEobj(pi[..., 4], tobj)
            lobj += obji * self.balance[i]  # obj loss
            if self.autobalance:
                self.balance[i] = self.balance[i] * 0.9999 + 0.0001 / obji.detach().item()

        if self.autobalance:
            self.balance = [x / self.balance[self.ssi] for x in self.balance]
        lbox *= self.lambda_box
        lobj *= self.lambda_obj
        lcls *= self.lambda_cls
        bs = tobj.shape[0]  # batch size

        return (lbox + lobj + lcls) * bs, torch.cat((lbox, lobj, lcls)).detach()

    def build_targets(self, p, targets):
        """Prepares model targets from input targets (image,class,x,y,w,h) for loss computation, returning class, box,
        indices, and anchors.
        """
        na, nt = self.na, targets.shape[0]  # number of anchors, targets
        tcls, tbox, indices, anch = [], [], [], []
        gain = torch.ones(7, device=self.device)  # normalized to gridspace gain
        ai = torch.arange(na, device=self.device).float().view(na, 1).repeat(1, nt)  # same as .repeat_interleave(nt)
        targets = torch.cat((targets.repeat(na, 1, 1), ai[..., None]), 2)  # append anchor indices

        g = 0.5  # bias
        off = (
        torch.tensor(
            [
                [0, 0],
                [1, 0],
                [0, 1],
                [-1, 0],
                [0, -1],  # j,k,l,m
                # [1, 1], [1, -1], [-1, 1], [-1, -1],  # jk,jm,lk,lm
            ],
            device=self.device,
        ).float()
        * g
    )  # offset
    
        for i in range(self.nl):
            anchors, shape = self.anchors[i], p[i].shape
            gain[2:6] = torch.tensor(shape)[[3, 2, 3, 2]]  # xyxy gain

            # Match targets to anchors
            t = targets * gain  # shape(3,n,7)

            if nt:
                # Matches
                r = t[..., 4:6] / anchors[:, None]  # wh ratio
                j = torch.max(r, 1 / r).max(2)[0] < anchor_t  # compare
                # j = wh_iou(anchors, t[:, 4:6]) > model.hyp['iou_t']  # iou(3,n)=wh_iou(anchors(3,2), gwh(n,2))
                t = t[j]  # filter

                # Offsets
                gxy = t[:, 2:4]  # grid xy
                gxi = gain[[2, 3]] - gxy  # inverse
                j, k = ((gxy % 1 < g) & (gxy > 1)).T
                l, m = ((gxi % 1 < g) & (gxi > 1)).T
                j = torch.stack((torch.ones_like(j), j, k, l, m))
                t = t.repeat((5, 1, 1))[j]
                offsets = (torch.zeros_like(gxy)[None] + off[:, None])[j]
            else:
                t = targets[0]
                offsets = 0

            # Define
            bc, gxy, gwh, a = t.chunk(4, 1)  # (image, class), grid xy, grid wh, anchors
            a, (b, c) = a.long().view(-1), bc.long().T  # anchors, image, class
            gij = (gxy - offsets).long()
            gi, gj = gij.T  # grid indices

            # Append
            indices.append((b, a, gj.clamp(0, shape[2] - 1), gi.clamp(0, shape[3] - 1)))  # image, anchor, grid
            tbox.append(torch.cat((gxy - gij, gwh), 1))  # box
            anch.append(anchors[a])  # anchors
            tcls.append(c)  # class

        return tcls, tbox, indices, anch

Compare original YOLOv5 loss with modified version (should output the same)

In [35]:
ultralytics_loss = ComputeLoss(anchor_t=anchor_t, balance=balance, strides=strides, lambda_box=lambda_box, lambda_cls=lambda_cls, lambda_obj=lambda_obj)
cleaned_loss = YOLOv5Loss(anchors, anchor_t=anchor_t, balance=balance, lambda_box=lambda_box, lambda_cls=lambda_cls, lambda_obj=lambda_obj)

Check that cleaned YOLOv5 loss version outputs the same results as the original one

In [36]:
# Build targets
print("=============== YOLOv5 ===============")
tcls, tbox, indices, anch = ultralytics_loss.build_targets(p, targets)
tcls2, tbox2, indices2, anchors2 = cleaned_loss.build_targets(p, targets)

for i, (t1, t2) in enumerate(zip(tcls, tcls2)):
    assert len(t1) == len(t2), f"Number of built-targets doesn't match in layer P{i+3} -> {len(tcls)} - {len(tcls2)}"
print(f"Number of built-targets: P3:{len(tcls[0])} | P4:{len(tcls[1])} | P5:{len(tcls[2])}")

for i in range(len(p)):
    assert torch.equal(tcls[i], tcls2[i]), "Target classes don't match"
    assert torch.equal(tbox[i], tbox2[i]), "Target boxes don't match"
    for ind1, ind2 in zip(indices[i], indices2[i]):
        assert torch.equal(ind1, ind2), "Indices don't match"
    assert torch.equal(anch[i], anchors2[i]), "Anchors don't match"
print("[%] All built-targets match")
print("======================================")

# Compute loss
total_loss, loss_items = ultralytics_loss(p, targets)
total_loss2, loss_items2 = cleaned_loss(p, targets)

print("Ultralytics YOLOv5 loss:    ", total_loss, loss_items)
print("Cleaned YOLOv5 loss version:", total_loss2, loss_items2)

assert torch.equal(total_loss, total_loss2), "Total loss doesn't match"
assert torch.equal(loss_items, loss_items2), "Loss items don't match"
print("[%] All losses match")
print("======================================")

=============== YOLOv5 ===============
Number of built-targets: P3:30 | P4:21 | P5:12
[%] All built-targets match
Ultralytics YOLOv5 loss:     tensor([7.7861]) tensor([0.1374, 3.0279, 0.7277])
Cleaned YOLOv5 loss version: tensor([7.7861]) tensor([0.1374, 3.0279, 0.7277])
[%] All losses match


Test YOLOv3 loss implementation and compare results with YOLOv5

> YOLOv3 creates much less built-targets than YOLOv5

In [37]:
yolov3_loss = YOLOv3Loss(anchors,  iou_t=0.2, lambda_box=lambda_box, lambda_cls=lambda_cls, lambda_obj=lambda_obj)

print("=============== YOLOv3 ===============")
tcls3, *_ = yolov3_loss.build_targets(p, targets)
print(f"Number of built-targets: P3:{len(tcls3[0])} | P4:{len(tcls3[1])} | P5:{len(tcls3[2])}")

total_loss3, loss_items3 = yolov3_loss(p, targets)
print("Custom YOLOv3 loss:", total_loss3, loss_items3)
print("======================================")

=============== YOLOv3 ===============
Number of built-targets: P3:3 | P4:3 | P5:2
Custom YOLOv3 loss: tensor([8.2952]) tensor([0.2516, 2.0245, 1.1832, 0.6883])
